In [ ]:
library(arrow)
library(dplyr)
library(mgcv)
library(tidyverse)
library(data.table)
library(metagam)
library(latex2exp)
library(eivtools)
library(plotly)
library(stats)
library(plotrix)
library(fishmethods)
library(metafor)

In [ ]:
plot_fit2<-function(df2, gamobj, coeff){
    
    df2<- df2 %>% mutate(site = substr(siteyr,1,nchar(siteyr)-4))
    #    df2<-df2 %>%
    #        mutate(lwr = df2[[coeff]] - 2 * df2[[paste0(coeff,'_var')]],
    #         upr = df2[[coeff]] + 2 * df2[[paste0(coeff,'_var')]])    

    fit1_FIT<-predict(gamobj, se.fit=T) %>%
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)

    ggplot()+
    geom_point(aes(df2[[coeff]], df2$auc, color=df2$site))+
    geom_line(aes(df2[[coeff]], fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2[[coeff]], ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")
#    geom_errorbarh(aes(y=df2$auc, xmin=df2$lwr, xmax=df2$upr), alpha = 0.2)    
}

In [ ]:
stg <- 'stg23'
fs <- 'nofs'
ckd_group <- 0

#path <- "metadata_mean.parquet"
#df <- arrow::read_parquet(path)
path <- paste0("/home/hoyinchan/blue/Data/data2021/data2021/",'metadata_',stg,'_',fs,'_.parquet')
df <- arrow::read_parquet(path)
path <- paste0("/home/hoyinchan/blue/Data/data2021/data2021/",'metadata_raw_',stg,'_',fs,'_.parquet')
dfraw <- arrow::read_parquet(path)

In [ ]:
#df <- df %>% filter(ckd_group == 1)
#dfraw <- dfraw %>% filter(ckd_group == 0)

In [ ]:
df = df %>% dplyr::select(-'__index_level_0__') %>% mutate(siteyr=paste0(site,year)) %>% filter(Importances!=0)

#df = df %>% filter(!(site %in% list("MCRI"))) %>% filter(!(siteyr %in% list("UMHC2013", "UMHC2020")))
df = df %>% filter(!(siteyr %in% list("UMHC2020", "MCRI2016", "MCRI3000")))

siteyrlist = df %>% dplyr::select(siteyr) %>% unique
featuredecodelist = df %>% dplyr::select(featuredecode) %>% unique
dfraw = dfraw %>% rename(fval=Name) %>% rename(siteyr=siteyear) %>% dplyr::select(-'__index_level_0__') %>% filter(!(siteyr %in% list("UMHC2020", "MCRI2016", "MCRI3000")))

In [ ]:
filterdf <- df %>% 
    select(site,year,featuredecode,fval) %>% 
    group_by(site,year,featuredecode) %>% 
    summarise(min=min(fval), max=max(fval)) %>%
    ungroup %>%
    group_by(featuredecode) %>%
    summarise(min_min=min(min), max_min=max(min), min_max=min(max), max_max=max(max), mean_min=mean(min), sd_min=sd(min), mean_max=mean(max), sd_max=sd(max)) %>%
    mutate(upr=mean_max+3*sd_max, lwr=mean_min-3*sd_min) %>% 
    mutate(upr = replace(upr, featuredecode == "AGE", 89)) %>% 
    mutate(lwr = replace(lwr, featuredecode == "AGE", 18))

df <- df %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)
dfraw <- dfraw %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)

In [ ]:
frange <- dfraw %>% select(fval, featuredecode) %>% group_by(featuredecode) %>% 
                        summarize(min=min(fval),q025 =quantile(fval,probs=0.025),
                                                q25 =quantile(fval,probs=0.25),
                                                q375=quantile(fval,probs=0.375),
                                                q50 =quantile(fval,probs=0.50),
                                                q625=quantile(fval,probs=0.625),
                                                q75 =quantile(fval,probs=0.75),
                                                q975 =quantile(fval,probs=0.975),
                                                max=max(fval))
#frange

In [ ]:
# #fit curve at theior own domain
# ptm <- proc.time()
# gamdict <- list()
# for (sy in siteyrlist$siteyr){
#     dfrawsy <- dfraw %>% filter(siteyr==sy)
#     syflist <- dfrawsy %>% select(featuredecode) %>% unique
#     gamsydict <- list()
#     for (syf in syflist$featuredecode){
# #        print(paste0(sy,',',syf))
# #        flush.console()
#         dftt <- dfrawsy %>% filter(featuredecode==syf)
#         if (!all(dftt['val']==0)){
#             x<-list(gam(val ~ s(fval, k=6, bs='cs'), data=dftt, method='REML'))
#             names(x) <- syf
#             gamsydict <- append(gamsydict,x)
#         }
#     }
#     gamsydict2<-list(gamsydict)
#     names(gamsydict2) <- sy
#     gamdict <- append(gamdict,gamsydict2)
# }
# proc.time() - ptm

# saveRDS(gamdict, file = "gamfitobj.rds")
# #gamdict<-readRDS(file = "gamfitobj.rds")

# #Get the value at equal distance
# fitlist = list()
# for (sy in siteyrlist$siteyr){
#     for (syf in frange$featuredecode){
#         fr<-frange %>% filter(featuredecode==syf)
#         newd<- data.frame(fval=seq(from=fr$min, to=fr$max, length.out=10))
#         x<-gamdict[[sy]][[syf]]
#         if (!is.null(x)){
#             fit1_FIT <- list(data.frame(predict(x, newd, se.fit=T) %>% as_tibble() %>% mutate(siteyr=sy,featuredecode=syf),newd))
#             fitlist <- append(fitlist,fit1_FIT)
#         }
#     }
# }
# fitlist<-rbindlist(fitlist) %>% mutate(fval=round(fval,2)) %>% mutate(fval=paste0(featuredecode,'.',fval))
# fitlist<-fitlist %>% mutate(featuredecode=str_replace(featuredecode,'-','.')) %>% mutate(featuredecode=paste0('f',featuredecode))
# fitlist<-fitlist %>% mutate(fval=str_replace(fval,'-','.')) %>% mutate(fval=paste0('f',fval))

In [ ]:
#drop not working feature
dfraw<-dfraw %>% filter(featuredecode!='99232')  %>% filter(featuredecode!='585bt6') 

In [ ]:
# syf = 'AGE'
# sy = 'UTSW3000'

#     siteyr <- list()
#     fits <- list() 
#     dftt <- dfraw %>% filter(featuredecode==syf)

#     fr<-frange %>% filter(featuredecode==syf)

#                 dfrawsy <- dftt %>% filter(siteyr==sy)                
#                 dfrawsy <- dfrawsy %>% filter(fval>=fr$q025) %>% filter(fval<=fr$q975)                
#                 xfit <- gam(val ~ s(fval, k=10, bs='cr'), data=dfrawsy, method='REML')
#                 xfit2 <- strip_rawdata(xfit)
#                 fits <- c(fits, list(xfit2))
#                 siteyr <- c(siteyr, sy)
                
#                 dftta <- df %>% filter(featuredecode==syf)
#                 dfrawsya <- dftta %>% filter(siteyr==sy)                
#                 dfrawsya <- dfrawsya %>% filter(fval>=fr$q025) %>% filter(fval<=fr$q975)                
#                 fcplot <- ggplot() + 
#                 geom_point(data=dfrawsya, aes(x=fval,y=mean_val)) +
#                 geom_errorbar(data=dfrawsya, aes(x=fval,y=mean_val,ymin = mean_val+qnorm(.025)*se_val, ymax = mean_val+qnorm(.975)*se_val)) +
#                 geom_line(data=pred_data, aes(x=fval, y=fit, color='r'))                
#                 ggsave(file=paste0(syf,'_',sy,"_check.svg"), plot=fcplot, width=10, height=8)                                    
    

#         grid <- data.frame(fval = seq(from=fr$q025, to=fr$q975, length = 100))
#          grid2 <- grid
#          pred <- predict(xfit2, newdata = grid2, se.fit = TRUE,
#          type = "iterms", terms = c("s(fval)"))
#          pred_data <- grid2 %>%
#          mutate(
#          fit = rowSums(pred$fit) + attr(pred, "constant"),
#          se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))))

#         # ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, color='r')) + 
#         #     geom_line() + 
#         #     geom_hline(yintercept=0) +
#         #     geom_ribbon(alpha = .3, color = NA) +
#         #     theme_classic() +
#         #     theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
#         #     ylab("SHAP") +
#         #     xlab(paste0(syf,'-',sy)) +
#         #     geom_point(data=dfrawsy, aes(x=fval,y=val))
                    
                    
#     # ggplot()+geom_point(data=dfrawsy, aes(x=fval,y=val))+geom_line(data=pred_data, aes(x=fval, y=fit, color='r')) +
#     #         geom_hline(yintercept=0) +
#     #         theme_classic() +
#     #         theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
#     #         ylab("SHAP") +
#     #         xlab(paste0(syf,'-',sy))
                    
#     # ggplot() + 
#     # geom_point(data=dfrawsya, aes(x=fval,y=mean_val)) + 
#     # geom_errorbar(data=dfrawsya, aes(x=fval,y=mean_val,ymin = mean_val+qnorm(.025)*se_val, ymax = mean_val+qnorm(.975)*se_val)) + 
#     # geom_line(data=pred_data, aes(x=fval, y=fit, color='r'))        
# #                        dfrawsya
#     dfcheck <- dfrawsy %>% group_by(fval) %>% summarise(mval=mean(val))
                    
#     ggplot() + 
#     geom_point(data=dfrawsya, aes(x=fval,y=mean_val)) +
#     geom_errorbar(data=dfrawsya, aes(x=fval,y=mean_val,ymin = mean_val+qnorm(.025)*se_val/sqrt(valsize), ymax = mean_val+qnorm(.975)*se_val/sqrt(valsize))) +
#     geom_line(data=pred_data, aes(x=fval, y=fit, color='r'))

In [ ]:
# #domanance plot
# library(metagam)
# gamdict <- list()
# ckd_group<-0
# syflist <- dfraw %>% select(featuredecode) %>% unique

# # library(doParallel)
# # myCluster <- makeCluster(10) # number of cores to use
# # registerDoParallel(myCluster)

# for (syf in syflist$featuredecode){
# #foreach(syf = syflist$featuredecode, .combine = 'c') %dopar% {
#     print(paste0(syf))
#     flush.console()    
#     siteyr <- list()
#     fits <- list() 
#     dftt <- dfraw %>% filter(featuredecode==syf)
#     fr<-frange %>% filter(featuredecode==syf)

#     dftta <- df %>% filter(featuredecode==syf)
    
# #    pplot <- ggplot(data=dftt, aes(x=fval, y=val)) + geom_point()
# #    ggsave(file=paste0(syf,"_point.svg"), plot=pplot, width=10, height=8)                    
    
#     tryCatch({        
#         for (sy in siteyrlist$siteyr){
#             tryCatch({                    
#                 dfrawsy <- dftt %>% filter(siteyr==sy)                
#                 dfrawsy <- dfrawsy %>% filter(fval>=fr$q025) %>% filter(fval<=fr$q975)                
#                 xfit <- gam(val ~ s(fval, k=10, bs='cr'), data=dfrawsy, method='REML')
#                 xfit2 <- strip_rawdata(xfit)
#                 fits <- c(fits, list(xfit2))
#                 siteyr <- c(siteyr, sy)
                
#                 dfrawsya <- dftta %>% filter(siteyr==sy)                
#                 dfrawsya <- dfrawsya %>% filter(fval>=fr$q025) %>% filter(fval<=fr$q975)                

#                 grid <- data.frame(fval = seq(from=fr$q025, to=fr$q975, length = 100))
#                  grid2 <- grid
#                  pred <- predict(xfit2, newdata = grid2, se.fit = TRUE,
#                  type = "iterms", terms = c("s(fval)"))
#                 pred_data <- grid2 %>%
#                  mutate(
#                  fit = rowSums(pred$fit) + attr(pred, "constant"),
#                  se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))))
                
#                 fcplot <- ggplot() + 
#                 geom_point(data=dfrawsya, aes(x=fval,y=mean_val)) +
#                 geom_errorbar(data=dfrawsya, aes(x=fval,y=mean_val,ymin = mean_val+qnorm(.025)*se_val/sqrt(valsize), ymax = mean_val+qnorm(.975)*se_val/sqrt(valsize))) +
#                 geom_line(data=pred_data, aes(x=fval, y=fit, color='r'))                
#                 ggsave(file=paste0(syf,'_',sy,"_check_",ckd_group,".svg"), plot=fcplot, width=10, height=8)                                    
#             },
#             error = function(e) print(paste0(syf,' ',sy)))                
#         }

#         grid <- data.frame(fval = seq(from=fr$q025, to=fr$q975, length = 100))
#         metafit <- metagam(fits, grid, terms = "s(fval)", method = "DL", intercept = "TRUE")

#         fplot2 <- plot(metafit, legend=TRUE, ci='both')
# #        fplot2 <- fplot + labs(x=syf)
        
#         dplot <- plot_dominance(metafit)
#         dplot2 <- dplot + labs(x=syf) + scale_fill_discrete(labels = siteyr)

#         hplot <- plot_heterogeneity(metafit)
#         hplot2 <- hplot + labs(x=syf)

#         ggsave(file=paste0(syf,"_meta_",ckd_group,".svg"), plot=fplot2, width=10, height=8)        
#         ggsave(file=paste0(syf,"_dominance_",ckd_group,".svg"), plot=dplot2, width=10, height=8)
#         ggsave(file=paste0(syf,"_heterogeneity_",ckd_group,".svg"), plot=hplot2, width=10, height=8)
        
#         cohort_fits <- fits

#         pred_data <- imap_dfr(cohort_fits, function(fit, cohort){

#          lower <- min(fit$var.summary$fval)
#          upper <- max(fit$var.summary$fval)

#          grid2 <- grid

#          pred <- predict(fit, newdata = grid2, se.fit = TRUE,
#          type = "iterms", terms = c("s(fval)"))

#          grid2 %>%
#          mutate(
#          fit = rowSums(pred$fit) + attr(pred, "constant"),
#          se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))),
#          cohort = siteyr[[cohort]]
#         # cohort = cohort                
#          )
#         })

#         pred_data2 <- metafit$meta_estimates %>% select('fval', 'estimate', 'se') %>% dplyr::rename(fit=estimate) %>% mutate(cohort='meta')
#         pred_data <- bind_rows(pred_data, pred_data2)

#         mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=cohort, color = cohort, fill = cohort)) + 
#             geom_line() + 
#             geom_hline(yintercept=0) +
#             geom_ribbon(alpha = .3, color = NA) +
#             theme_classic() +
#             theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
#             ylab("SHAP") +
#             xlab(syf)    

#         ggsave(file=paste0(syf,"_meta2_",ckd_group,".svg"), plot=mplot, width=10, height=8)                

            
#     },
#     error = function(e) print(e))
# }

In [ ]:
# #domanance plot
library(metagam)

custom_metaplot <- function(syf, minfval, maxfval, knots, ckd_group, stg, fs) {
    print(paste0(syf))
    flush.console()    
    siteyr <- list()
    fits <- list() 
    dftt <- dfraw %>% filter(featuredecode==syf)
    fr<-frange %>% filter(featuredecode==syf)

    dftta <- df %>% filter(featuredecode==syf)
    
    pplot <- ggplot(data=dftt, aes(x=fval, y=val)) + geom_point()
#    ggsave(file=paste0(syf,"_point.svg"), plot=pplot, width=10, height=8)                    
    ggsave(file=paste0("../plots/",syf,"_point.jpg"), plot=pplot, width=10, height=8)                        
    return
    
    tryCatch({        
        for (sy in siteyrlist$siteyr){
            tryCatch({                    
                dfrawsy <- dftt %>% filter(siteyr==sy)                
                dfrawsy <- dfrawsy %>% filter(fval>=minfval) %>% filter(fval<=maxfval)                
                xfit <- gam(val ~ s(fval, k=knots, bs='cr'), data=dfrawsy, method='REML')
                xfit2 <- strip_rawdata(xfit)
                fits <- c(fits, list(xfit2))
                siteyr <- c(siteyr, sy)
                
                dfrawsya <- dftta %>% filter(siteyr==sy)                
                dfrawsya <- dfrawsya %>% filter(fval>=minfval) %>% filter(fval<=maxfval)                

                grid <- data.frame(fval = seq(from=minfval, to=maxfval, length = 100))
                 grid2 <- grid
                 pred <- predict(xfit2, newdata = grid2, se.fit = TRUE,
                 type = "iterms", terms = c("s(fval)"))
                pred_data <- grid2 %>%
                 mutate(
                 fit = rowSums(pred$fit) + attr(pred, "constant"),
                 se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))))
                
                fcplot <- ggplot() + 
                geom_point(data=dfrawsya, aes(x=fval,y=mean_val)) +
                geom_errorbar(data=dfrawsya, aes(x=fval,y=mean_val,ymin = mean_val+qnorm(.025)*se_val/sqrt(valsize), ymax = mean_val+qnorm(.975)*se_val/sqrt(valsize))) +
                geom_line(data=pred_data, aes(x=fval, y=fit, color='r'))                
                ggsave(file=paste0("../plots/",syf,'_',sy,"_check_",ckd_group,".svg"), plot=fcplot, width=10, height=8)                                    
            },
            error = function(e) print(paste0(syf,' ',sy)))                
        }
    
        grid <- data.frame(fval = seq(from=minfval, to=maxfval, length = 100))
        metafit <- metagam(fits, grid, terms = "s(fval)", method = "DL", intercept = "TRUE")

        fplot2 <- plot(metafit, legend=TRUE, ci='both')
#        fplot2 <- fplot + labs(x=syf)
        
        dplot <- plot_dominance(metafit)
        dplot2 <- dplot + labs(x=syf) + scale_fill_discrete(labels = siteyr)

        hplot <- plot_heterogeneity(metafit)
        hplot2 <- hplot + labs(x=syf)

        ggsave(file=paste0("../plots/",syf,"_meta_",ckd_group,"_",stg,"_",fs,".svg"), plot=fplot2, width=10, height=8)        
        ggsave(file=paste0("../plots/",syf,"_dominance_",ckd_group,"_",stg,"_",fs,".svg"), plot=dplot2, width=10, height=8)
        ggsave(file=paste0("../plots/",syf,"_heterogeneity_",ckd_group,"_",stg,"_",fs,".svg"), plot=hplot2, width=10, height=8)
        
        cohort_fits <- fits

        pred_data <- imap_dfr(cohort_fits, function(fit, cohort){

         lower <- min(fit$var.summary$fval)
         upper <- max(fit$var.summary$fval)

         grid2 <- grid

         pred <- predict(fit, newdata = grid2, se.fit = TRUE,
         type = "iterms", terms = c("s(fval)"))

         grid2 %>%
         mutate(
         fit = rowSums(pred$fit) + attr(pred, "constant"),
         se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))),
         cohort = siteyr[[cohort]]
        # cohort = cohort                
         )
        })

        pred_data2 <- metafit$meta_estimates %>% select('fval', 'estimate', 'se') %>% dplyr::rename(fit=estimate) %>% mutate(cohort='meta')
        pred_data <- bind_rows(pred_data, pred_data2)

        mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=cohort, color = cohort, fill = cohort)) + 
            geom_line() + 
            geom_hline(yintercept=0) +
            geom_ribbon(alpha = .3, color = NA) +
            theme_classic() +
            theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
            ylab("SHAP") +
            xlab(syf)    

        save(pred_data2,file=paste0("../plots/",syf,"_meta2_",ckd_group,"_",stg,"_",fs,".Rdata"))        
        ggsave(file=paste0("../plots/",syf,"_meta2_",ckd_group,"_",stg,"_",fs,".svg"), plot=mplot, width=10, height=8)                
    },
    error = function(e) print(e))
}
 
knots <- 10       
syflist <- dfraw %>% select(featuredecode) %>% unique                                                            
for (syf in syflist$featuredecode){
    print(paste0(syf))
    flush.console()    
    siteyr <- list()
    fits <- list() 
    dftt <- dfraw %>% filter(featuredecode==syf)
    fr<-frange %>% filter(featuredecode==syf)
    dftta <- df %>% filter(featuredecode==syf)
    custom_metaplot(syf, fr$q025, fr$q975, knots, ckd_group, stg, fs)                                                           
}                              
                              
#custom_metaplot('2823-3', 1.5, 5, 10)
#custom_metaplot('2951-2', 1.5, 5, 10)

In [ ]:
custom_metaplot_combine <- function(syf, minfval, maxfval, stg, fs) {

    tryCatch({    
#        pred_data_env <- load(file=paste0(syf,"_meta2_",1,"_",stg,"_",fs,".Rdata")) 
        pred_data_env <- load(file=paste0(syf,"_meta2_",1,".Rdata"))         
        pred_data <- get(pred_data_env) %>% mutate(ckd_group_col=as.character(1))          
        for (ckd_group in 2:4){
            tryCatch({
#                pred_data2_env <- load(file=paste0(syf,"_meta2_",ckd_group,"_",stg,"_",fs,".Rdata"))       
                pred_data2_env <- load(file=paste0(syf,"_meta2_",ckd_group,".Rdata"))                       
                pred_data2 <- get(pred_data2_env) %>% mutate(ckd_group_col=as.character(ckd_group))
                pred_data <- bind_rows(pred_data, pred_data2)
                },
             error = function(e) print(paste0('Missing ',syf,' ',sy)))                            
        }
        mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=ckd_group_col, color = ckd_group_col, fill = ckd_group_col)) + 
            geom_line() + 
            geom_hline(yintercept=0) +
            # geom_ribbon(alpha = .3, color = NA) +
            theme_classic() +
            theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
            ylab("SHAP") +
            xlab(syf)    
        ggsave(file=paste0("../plots/",syf,"_meta2_ckd_",stg,"_",fs,".svg"), plot=mplot, width=10, height=8)                    
        },
        error = function(e) print(paste0('Missing ',syf,' meta')))                                    
}

#syflist <- dfraw %>% select(featuredecode) %>% unique                                                            
syf = '17861-6'
#for (syf in syflist$featuredecode){                            
    print(paste0(syf))
    flush.console()    
    siteyr <- list()
    fits <- list() 
    dftt <- dfraw %>% filter(featuredecode==syf)
    fr<-frange %>% filter(featuredecode==syf)
    dftta <- df %>% filter(featuredecode==syf)
    custom_metaplot_combine(syf, fr$q025, fr$q975, stg, fs)                                                           
#}                                                          
#custom_metaplot_combine('AGE', 18, 80)

In [ ]:
custom_custom_metaplot_combine <- function(syf,syfname,minfval,maxfval,lhline,hhline) {

 #   tryCatch({    
        pred_data_env <- load(file=paste0(syf,"_meta2_",0,".Rdata")) 
        pred_data <- get(pred_data_env) %>% mutate(ckd_group_col=as.character(0))          
        for (ckd_group in 1:4){
            tryCatch({
                pred_data2_env <- load(file=paste0(syf,"_meta2_",ckd_group,".Rdata"))       
                pred_data2 <- get(pred_data2_env) %>% mutate(ckd_group_col=as.character(ckd_group))
                pred_data <- bind_rows(pred_data, pred_data2)
                },
             error = function(e) print(paste0('Missing ',syf,' ',sy)))                            
        }
        if (lhline == -1){             
            mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=ckd_group_col, color = ckd_group_col, fill = ckd_group_col)) + 
                geom_line() + 
                geom_hline(yintercept=0) +
                # geom_ribbon(alpha = .3, color = NA) +
                theme_classic() +
                theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
                xlim(minfval, maxfval) + 
                ggtitle(syfname) +         
                ylab("SHAP") +
                xlab(syf)    
        } else {
            mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=ckd_group_col, color = ckd_group_col, fill = ckd_group_col)) + 
                geom_line() + 
                geom_hline(yintercept=0) +
                geom_vline(xintercept=as.numeric(lhline), linetype="dotted") +
                geom_vline(xintercept=as.numeric(hhline), linetype="dotted") +                                 
                # geom_ribbon(alpha = .3, color = NA) +
                theme_classic() +
                theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
                xlim(minfval, maxfval) +             
                ggtitle(syfname) +         
                ylab("SHAP") +
                xlab(syf)                
        }
        ggsave(file=paste0("../plots/",syf,"_meta2_ckd.svg"), plot=mplot, width=10, height=8)                    
#        },
#        error = function(e) print(paste0('Missing ',syf,' meta')))                                    
}

nomrange <- data.frame("featuredecode" = "DIASTOLIC", "Name" = "Diastolic", "lrange" = 60, "hrange" = 80)
nomrange[nrow(nomrange)+1,] <- c("ORIGINAL_BMI", "BMI", 18.5, 24.9)                                         
nomrange[nrow(nomrange)+1,] <- c("6768-6" ,"Alkaline,phosphatase"   ,30,120)
nomrange[nrow(nomrange)+1,] <- c("787-2"  ,"Mean,corpuscular,volume",-1,-1)
nomrange[nrow(nomrange)+1,] <- c("19123-9","Magnesium"              ,-1,-1)
nomrange[nrow(nomrange)+1,] <- c("2823-3" ,"Potassium"              ,3.5,5)
nomrange[nrow(nomrange)+1,] <- c("2951-2" ,"Sodium"                 ,136,145)
nomrange[nrow(nomrange)+1,] <- c("17861-6","Calcium"                ,8.5,10.2)
nomrange[nrow(nomrange)+1,] <- c("AGE"    ,"Age"                    ,-1,-1)
nomrange[nrow(nomrange)+1,] <- c("2028-9"  ,"Carbon Dioxide"        ,21,30)
nomrange[nrow(nomrange)+1,] <- c("2075-0"  ,"Chloride"              ,98,106)
                     
#nomrange[nrow(nomrange)+1,] <- c("AGE","Age"                        ,-1,-1)
#nomrange[nrow(nomrange)+1,] <- c("AGE","Age"                        ,-1,-1)

#nomrange                    
syf = '17861-6'
#for (syf in nomrange$featuredecode){
    print(syf)
    fr<-frange %>% filter(featuredecode==syf)
    fn<-nomrange %>% filter(featuredecode==syf)                     
    custom_custom_metaplot_combine(syf, fn$Name, fr$q025, fr$q975, fn$lrange, fn$hrange)
    if (syf == '2823-3') {custom_custom_metaplot_combine(syf, fn$Name, 2, fr$q975, fn$lrange, fn$hrange)}    
    if (syf == 'SYSTOLIC') {custom_custom_metaplot_combine(syf, fn$Name, fr$min, fr$q975, fn$lrange, fn$hrange)}        
    if (syf == '2075-0') {custom_custom_metaplot_combine(syf, fn$Name, fr$q025, 120, fn$lrange, fn$hrange)}            
#}
                     

In [ ]:
#dfraw %>% select(featuredecode) %>% unique 

In [ ]:
input <- list('AGE', 18, 89)

syf = input[[1]]
grid_min <- input[[2]]
grid_max <- input[[3]]

siteyr <- list()
fits <- list() 
dftt <- dfraw %>% filter(featuredecode==syf)
fr<-frange %>% filter(featuredecode==syf)

for (sy in siteyrlist$siteyr){
    print(paste0(sy))
    flush.console()            
    dfrawsy <- dftt %>% filter(siteyr==sy)
    xfit <- gam(val ~ s(fval, k=6, bs='cr'), data=dfrawsy, method='REML')
    xfit2 <- strip_rawdata(xfit)
    fits <- c(fits, list(xfit2))
    siteyr <- c(siteyr, sy)
}

#grid <- data.frame(fval = seq(from=fr$min, to=fr$max, length = 20))
grid <- data.frame(fval = seq(from=grid_min, to=grid_max, length = 20))
metafit <- metagam(fits, grid, terms = "s(fval)", method = "DL", intercept = "TRUE")

cohort_fits <- fits

pred_data <- imap_dfr(cohort_fits, function(fit, cohort){

 print(cohort)    
 lower <- min(fit$var.summary$fval)
 upper <- max(fit$var.summary$fval)

 grid2 <- grid
    
 pred <- predict(fit, newdata = grid2, se.fit = TRUE,
 type = "iterms", terms = c("s(fval)"))

 grid2 %>%
 mutate(
 fit = rowSums(pred$fit) + attr(pred, "constant"),
 se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))),
 cohort = siteyr[[cohort]]
# cohort = cohort                
 )
})
  
pred_data2 <- metafit$meta_estimates %>% select('fval', 'estimate', 'se') %>% dplyr::rename(fit=estimate) %>% mutate(cohort='meta')
pred_data <- bind_rows(pred_data, pred_data2)
    
mplot <- ggplot(data=pred_data, aes(x=fval, y=fit, ymin = fit+qnorm(.025)*se, ymax = fit+qnorm(.975)*se, group=cohort, color = cohort, fill = cohort)) + 
    geom_line() + 
    geom_hline(yintercept=0) +
    geom_ribbon(alpha = .3, color = NA) +
    theme_classic() +
    theme(strip.background = element_blank(), panel.border = element_rect(colour = "black", fill = NA), panel.grid.major = element_line(size = 0.5, linetype = 2)) +
    ylab("SHAP") +
    xlab(syf)    
    
ggsave(file=paste0("../plots/",syf,"_meta2.svg"), plot=fplot2, width=10, height=8)        

In [ ]:
pplot <- ggplot(data=dfrawsy, aes(x=fval, y=val)) + geom_point()
ggsave(file=paste0("../plots/",syf,"_point.svg"), plot=fplot2, width=10, height=8)        

In [ ]:
# #domanance plot per site
# library(metagam)
# gamdict <- list()
# #syf = 'AGE'

# sitelist <- dfraw %>% dplyr::select(site) %>% unique
# syflist <- dfraw %>% select(featuredecode) %>% unique
# sitelist <- sitelist %>% filter(site!='MCRI')

# for (syf in syflist$featuredecode){
#     print(paste0(syf))
#     flush.console()    
#     site <- list()
#     fits <- list() 
#     dftt <- dfraw %>% filter(featuredecode==syf)
#     fr<-frange %>% filter(featuredecode==syf)

#     tryCatch({
#         for (sy in sitelist$site){
#             dfrawsy <- dftt %>% filter(site==sy)
#             xfit <- gam(val ~ s(fval, k=6, bs='cr'), data=dfrawsy, method='REML')
#             xfit2 <- strip_rawdata(xfit)
#             fits <- c(fits, list(xfit2))
#             site <- c(site, sy)
#         }

#         grid <- data.frame(fval = seq(from=fr$min, to=fr$max, length = 20))
#         metafit <- metagam(fits, grid, terms = "s(fval)", method = "DL", intercept = "TRUE")

#         dplot <- plot_dominance(metafit)
#         dplot2 <- dplot + labs(x=syf) + scale_fill_discrete(labels = site)

#         hplot <- plot_heterogeneity(metafit)
#         hplot2 <- hplot + labs(x=syf)

#         ggsave(file=paste0(syf,"_dominance_site2.svg"), plot=dplot2, width=10, height=8)
#         ggsave(file=paste0(syf,"_heterogeneity_site2.svg"), plot=hplot2, width=10, height=8)
#     },
#     error = function(e) e)
# }

In [ ]:
#Alternatively fit and get coefficient

#Make Average to be zero?
#dfraw <- dfraw %>% group_by(siteyr, Feature) %>% mutate(val=val-mean(val)) %>% ungroup

ptm <- proc.time()
gamdict <- list()
fitlist <- list()
#sy <- 'KUMC2013'
#syf <- 'SYSTOLIC'
for (sy in siteyrlist$siteyr){
    # if (sy != 'MCRI2016'){
    #     next
    # }
#    site <- substr(sy, 0, nchar(sy)-4)    
#    if ((site %in% list("MCRI")) | sy %in% list("UMHC2013", "UMHC2020")){
#    if ((site %in% list("MCRI")) | sy %in% list("UMHC2013", "UMHC2020")){    
#        next
#    }    
    dfrawsy <- dfraw %>% filter(siteyr==sy)
    syflist <- dfrawsy %>% select(featuredecode) %>% unique
    gamsydict <- list()
    for (syf in syflist$featuredecode){
        print(paste0(sy,'-',syf))
        flush.console()
        dftt <- dfrawsy %>% filter(featuredecode==syf)
        fr<-frange %>% filter(featuredecode==syf)
        if (!all(dftt['val']==0)){
            x<-list(gam(val ~ s(fval, k=6, bs='cr'), data=dftt, method='REML', knots = list(fval = seq(from=fr$min, to=fr$max, length = 6))))
            names(x) <- syf
            gamsydict <- append(gamsydict,x)
            
            rangelist <- list()
            for (i in 1:length(x[[syf]]$smooth[[1]]$xp)){
                rangelist <- c(rangelist, paste0('r', x[[syf]]$smooth[[1]]$xp[i], 'r', x[[syf]]$smooth[[1]]$xp[i+1], 'r'))
            }
            rangelist <- rangelist[-length(rangelist)]
            rangelist <- c('intercept', rangelist)
            coeffdata <- cbind(data.frame(x[[syf]]$coefficient), data.frame(diag(x[[syf]]$Vc)), data.frame(transpose(rangelist)))
            colnames(coeffdata) <- c('fit', 'se.fit', 'fval')
            coeffdata <- coeffdata %>% mutate(siteyr=sy,featuredecode=syf)
            coeffdata <- coeffdata[-1,]
            coeffdata <- coeffdata %>% mutate(fval=paste0(featuredecode,'.',fval))
            row.names(coeffdata) <- NULL
            fitlist <- append(fitlist,list(coeffdata))            
        }
    }
#    gamsydict2<-list(gamsydict)
#    names(gamsydict2) <- sy
#    gamdict <- append(gamdict,gamsydict2)
}
proc.time() - ptm
#gamdict<-rbindlist(gamdict)
fitlist<-rbindlist(fitlist) %>% mutate(featuredecode=str_replace(featuredecode,'-','d')) %>% mutate(featuredecode=paste0('f',featuredecode)) %>% mutate(fval=str_replace_all(fval,'-','d')) %>% mutate(fval=paste0('f',fval))

In [ ]:
#fitlist %>% filter(siteyr=='MCRI2016')
#fitlist <- fitlist %>% filter(substr(siteyr,1,4)!='MCRI')

In [ ]:
fitlist

In [ ]:
# test plot
# sy <- 'UofU2017'
# syf <- '2823-3'
# dfrawsy <- dfraw %>% filter(siteyr==sy)
# syflist <- dfrawsy %>% select(featuredecode) %>% unique
# dftt <- dfrawsy %>% filter(featuredecode==syf)
# fr<-frange %>% filter(featuredecode==syf)
# x<-gam(val ~ s(fval, k=10, bs='cr'), data=dftt, method='REML', knots = list(fval = seq(from=fr$min, to=fr$max, length = 10)))
# plot.gam(x, residuals=TRUE, seWithMean=2, xlim=c(fr$min, fr$max))

In [ ]:
# Combining Variance as weight
fitlist <- fitlist %>% group_by(fval) %>% mutate(re.fit = var(se.fit)) %>% ungroup()
#fitlist <- fitlist %>% mutate(weight=re.fit/mweight)

#Use maximum Variance (Fail)
#fitlist<- fitlist %>% group_by(fval) %>% mutate(re.fit = max(se.fit)) %>% ungroup()
#Use combine Variance (Fail)
#library(fishmethods)
#fitlist<- fitlist %>% group_by(fval) %>% mutate(re.fit = combinVar(fit, se.fit)) %>% ungroup()

# Try to use meta analysis to combine variance (Fail)
# refitlist <- list()
# for (coeff in unique(fitlist$fval)){
#     fitlistX <- fitlist %>% filter(fval == coeff)
#     tdf <- data.frame(list(coeff, rma(fitlistX$fit, fitlistX$se.fit)$vb[[1]]))
#     row.names(tdf) <- NULL
#     colnames(tdf) <- c('fval', 're.fit')
#     refitlist <- append(refitlist,list(tdf))         
# }    
# refitlist <- rbindlist(refitlist)
# refitlist
# fitlistXX <- fitlist %>% filter(fval == 'fSYSTOLIC.r414.222222222222r466r')
# rma(fitlistXX$fit, fitlistXX$se.fit)
# fitlistXX

In [ ]:
#fitlist
#fdc <- 'f2823d3'
#fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,fit) %>% spread(fval,fit)

In [ ]:
# error-in-variance (variance too large)
# fitlisttest <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,fit) %>% spread(fval,fit)
# fitlisttestse <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,se.fit) %>% spread(fval,se.fit)

# auctable<-df %>% select(siteyr,auc) %>% unique
# fitlisttest <- fitlisttest %>% left_join(auctable,by='siteyr')
# fitlisttestse <- fitlisttestse %>% left_join(auctable,by='siteyr')
# metaformula<-as.formula(paste0("auc~", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = '+')))

# fitlistD <- fitlist %>% filter(featuredecode==fdc) %>% select(fval, re.fit) %>% unique
# Sigma_error <- diag(fitlistD$re.fit)
# dimnames(Sigma_error) <- list(fitlistD$fval, fitlistD$fval)
# eivreg(metaformula, data = fitlisttest, Sigma_error = Sigma_error)

In [ ]:
featuretest <- function(fdc){
    fitlisttest <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,fit) %>% spread(fval,fit)
    fitlisttestse <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,se.fit) %>% spread(fval,se.fit)

    auctable<-df %>% select(siteyr,auc) %>% unique
    fitlisttest <- fitlisttest %>% left_join(auctable,by='siteyr')
    fitlisttestse <- fitlisttestse %>% left_join(auctable,by='siteyr')
    metaformulastr <- paste0("auc~", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = '+'))  
#    metaformulastr <- str_replace(metaformulastr, "-", "n")
    metaformula<-as.formula(metaformulastr)
#    metagame<-gam(metaformula, data=fitlisttest, method='REML')
    metagame<-gam(metaformula, data=fitlisttest, method='REML')

    return(list(metagame, fitlisttest))
}
featuretest1D <- function(fdc){
    fitlisttest <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,fit) %>% spread(fval,fit)
    fitlisttestse <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,se.fit) %>% spread(fval,se.fit)

    auctable<-df %>% select(siteyr,auc) %>% unique
    fitlisttest <- fitlisttest %>% left_join(auctable,by='siteyr')
    fitlisttestse <- fitlisttestse %>% left_join(auctable,by='siteyr')
    metaformula<-as.formula(paste0("auc~", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = '+')))
    metagame<-gam(metaformula, data=fitlisttest, method='REML')

    return(list(metagame, fitlisttest))
}

In [ ]:
fealist <- (fitlist %>% select(featuredecode) %>% unique)$featuredecode
flist <- list()
for (f in fealist){
    if (f == 'f26464d8'){
        next
    }
    print(f)
    tr <- featuretest1D(f)
    pv <- summary(tr[[1]])$p.table[,4]
    flist <- c(flist, names(pv)[abs(pv)<=0.05][-1])
    print(summary(tr[[1]]))
}

In [ ]:
fitlist

In [ ]:
fdc = "fDIASTOLIC"
    fitlisttest <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,fit) %>% spread(fval,fit)
    fitlisttestse <- fitlist %>% filter(featuredecode==fdc) %>% select(fval,siteyr,se.fit) %>% spread(fval,se.fit)

    auctable<-df %>% select(siteyr,auc) %>% unique
    fitlisttest <- fitlisttest %>% left_join(auctable,by='siteyr')
    fitlisttestse <- fitlisttestse %>% left_join(auctable,by='siteyr')
    metaformula<-as.formula(paste0("auc~", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = '+')))
    metagame<-gam(metaformula, data=fitlisttest, method='REML')

In [ ]:
fitlisttest
metagame<-gam(metaformula, data=fitlisttest, method='REML')
summary(metagame)

In [ ]:
plot(metagame, all.terms=TRUE)

In [ ]:
summary(metagame)

In [ ]:
repeat{
    fitlisttest <- fitlist %>% filter(fval %in% flist) %>% select(fval,siteyr,fit) %>% spread(fval,fit)
    fitlisttestse <- fitlist %>% filter(fval %in% flist) %>% select(fval,siteyr,se.fit) %>% spread(fval,se.fit)

    auctable<-df %>% select(siteyr,auc) %>% unique
    fitlisttest <- fitlisttest %>% left_join(auctable,by='siteyr') %>% replace(is.na(.), 0)
    fitlisttestse <- fitlisttestse %>% left_join(auctable,by='siteyr') %>% replace(is.na(.), 0)

#     metaformula<-as.formula(paste0("auc~ ", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = '+')))
#     metagame<-gam(metaformula, data=fitlisttest, method='REML')
    
#     pv <- summary(metagame)$p.table[,4]
#     if (all(pv[-1] <= 0.05)){
#         metagamefinal <- metagame
#         break
#     }
#     flist <- flist[-which.max(pv[-1])]

   metaformula<-as.formula(paste0("auc~ s(", paste(names(fitlisttest)[-length(fitlisttest)][-1], collapse = ',k=5,bs=\'cs\')+s('),',k=5,bs=\'cs\')'))
   metagame<-gam(metaformula, data=fitlisttest, method='REML')

    pv <- summary(metagame)$s.table[,4]
    if (all(pv <= 0.05)){
        metagamefinal <- metagame
        break
    }
    flist <- flist[-which.max(pv)]
    
}

In [ ]:
summary(metagamefinal)

In [ ]:
plot.gam(metagamefinal, all.terms = TRUE)

In [ ]:
tt<-predict(metagamefinal,fitlisttest)
fitlisttest2 <- fitlisttest
fitlisttest2['tt'] <- tt
fitlisttest2 <- fitlisttest2 %>% mutate(yr = substr(siteyr, nchar(siteyr)-3, nchar(siteyr))) %>% mutate(site = substr(siteyr, 0, nchar(siteyr)-4))
ggplot(fitlisttest2, aes(x=auc, y=tt, label = site))+
geom_point() +
geom_abline(intercept = 0, slope = 1,color='red') +
geom_text(hjust=0, vjust=0)

In [ ]:
plot_fit4<-function(df2, gamobj, titleinput){
    fit1_FIT<-predict(gamobj, se.fit=T) %>%
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)

    ggplot()+ labs(title=titleinput)+ theme(plot.title = element_text(hjust = 0.5)) +
    geom_point(aes(df2$fval, df2$val))+
    geom_line(aes(df2$fval, fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2$fval, ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")
}
for (sy in siteyrlist$siteyr){
    dfrawsy <- dfraw %>% filter(siteyr==sy)
    for (syf in frange$featuredecode){
        dftt <- dfrawsy %>% filter(featuredecode==syf)
        modelfit<-gamdict[[sy]][[syf]]    
        if (!is.null(modelfit)){
            print(plot_fit4(dftt, modelfit,paste0(sy,',',syf)))
            flush.console()
        }
    }
}

In [ ]:
# Confidence Interval
#fitlisttest2 %>% mutate(diff = abs(auc-tt)) %>% group_by(yr) %>% summarize(mean(diff))
fitlisttest3 <- fitlisttest2 %>% mutate(diff = auc-tt) %>% arrange(desc(diff))
x <- as.list(fitlisttest3['diff'])$diff
ci <- 0.95
Margin_Error <- qt(ci + (1 - ci)/2, df = length(x) - 1) * sd(x)/sqrt(length(x))
Margin_Error

In [ ]:
plot(density(fitlisttest3$diff))

In [ ]:
fitlisttest2 %>% mutate(diff = abs(auc-tt)) %>% group_by(site) %>% summarize(mean(diff))